# Exploratory Anlysis of HC data

In [ ]:
#pip install geopandas
#pip install folium

In [ ]:
# module imports
import os

import matplotlib.pyplot as plt

import numpy as np

import folium
import geopandas as gpd
from folium.plugins import HeatMap

import pandas as pd
import json
from json import loads, dumps

In [ ]:
pd.set_option('display.max_colwidth', None)
path = "./HCAHPSPatientSurveyFiles/data_tables/"

In [ ]:
measures_df = pd.read_csv("./HCAHPSPatientSurveyFiles/data_tables/measures.csv")
national_results_df = pd.read_csv('./HCAHPSPatientSurveyFiles/data_tables/national_results.csv')
questions_df = pd.read_csv('./HCAHPSPatientSurveyFiles/data_tables/questions.csv')
reports_df = pd.read_csv('./HCAHPSPatientSurveyFiles/data_tables/reports.csv')
responses_df = pd.read_csv('./HCAHPSPatientSurveyFiles/data_tables/responses.csv')
states_df = pd.read_csv('./HCAHPSPatientSurveyFiles/data_tables/states.csv')
state_results_df = pd.read_csv('./HCAHPSPatientSurveyFiles/data_tables/state_results.csv')

In [ ]:
##For future reference put in list and iterate through list.  Might save on lines of code but unsure of speed
#Strip Whitespace from columns
measures_strip = measures_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
national_results_strip_df = national_results_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
questions_strip_df = questions_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
reports_strip_df = reports_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
responses_strip_df = responses_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
states_strip_df = states_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
state_results_strip_df = state_results_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [ ]:
# ##names = ['measures','national_results','questions','reports','responses','states','state_results']
# df_list = []
# os.chdir(path)
# for file in os.listdir():
#     if file.endswith(".csv"):
#         df_list.append(pd.read_csv(file))

In [ ]:
# measures_df = df_list[0]
# national_results_df = df_list[1]
# questions_df = df_list[2]
# reports_df = df_list[3]
# responses_df = df_list[4]
# states_df = df_list[5]
# state_results_df = df_list[6]

In [ ]:
responses_strip_df.head()

In [ ]:
responses_strip_df['Release Period'] = responses_strip_df['Release Period'].str.replace('_','/')


In [ ]:
responses_strip_df.dtypes

In [ ]:

##Remove rows and facility ID's that produced not available.  It is unclear why they are not available.
##Could be do to no participating, not sharing, or combination of other factors.

not_available = responses_strip_df[ (responses_strip_df['Response Rate (%)'] == 'Not Available')].index
responses_strip_df.drop(not_available , inplace=True)

responses_strip_df['Response Rate (%)'] = responses_strip_df['Response Rate (%)'].astype(np.int64)

In [ ]:
%matplotlib inline
outliers = responses_strip_df[responses_strip_df['Response Rate (%)']<28]
_=outliers['Response Rate (%)'].plot.hist(title="Less Than < 20")

In [ ]:
avg = np.mean(responses_strip_df['Response Rate (%)'])
sd = np.std(responses_strip_df['Response Rate (%)'])

# Find upper and lower bound for outliers
print('The average response rate is: "{0:0.2f}"'.format(avg))
print('The standard deviation of response rate is: "{0:0.2f}"'.format(sd))



In [ ]:
def normalize(df, pd_series_name, nsd=2):
    '''
    Take all values that are outside some bound (mean +- 2 sd by default)
    and convert them to the appropriate bound.
    '''
    df = df.copy()
    pd_series = df[pd_series_name].astype(float)

    # Find upper and lower bound for outliers
    avg = np.mean(pd_series)
    sd  = np.std(pd_series)

    # Calculate the bounds
    lower_bound = avg - nsd*sd
    upper_bound = avg + nsd*sd

    # Collapse in the outliers: replace them with appropriate bound
    df.loc[pd_series < lower_bound , pd_series_name ] = lower_bound
    df.loc[pd_series > upper_bound , pd_series_name ] = upper_bound
    
    return (df[pd_series_name] - avg) / sd

In [ ]:
responses_df_normalized = responses_strip_df.copy()

In [ ]:
responses_df_normalized['Responses Normalized'] = normalize(responses_strip_df,'Response Rate (%)')

In [ ]:
responses_df_normalized

In [ ]:
states_strip_df.head()

In [ ]:
state_results_strip_df.head()

In [ ]:
states_merged = pd.merge(
    states_strip_df, state_results_strip_df, how="outer", on=["State"]
)

In [ ]:
states_merged.head()

In [ ]:
response_merge_states = pd.merge(
    states_strip_df, responses_strip_df, how="outer", on=["State"])

In [ ]:
response_merge_states

In [ ]:
response_merge_states['Completed Surveys'].replace({"FEWER THAN 50": '25', "300 or more": '300',
                                                   "Between 100 and 299":'250',"Fewer than 100":'50'}, inplace=True)

In [ ]:
#convert Completed Surveys string to int64
response_merge_states['Completed Surveys'] = response_merge_states['Completed Surveys'].astype(np.int64)

In [ ]:
max_surveys = response_merge_states['Completed Surveys'].max()
min_survyes = response_merge_states['Completed Surveys'].min()
print(max_surveys,min_survyes)

In [ ]:
# Generate sample data
np.random.seed(42)  # to get consistent results
num_rows = 37445

states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA',
          'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
          'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
          'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
          'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY','PR']

sample_states = np.random.choice(states, num_rows)
sample_surveys = np.random.randint(25, 13731, num_rows)

df = pd.DataFrame({
    'state': sample_states,
    'completed_surveys': sample_surveys
})

# Bar plot for 'state'
plt.figure(figsize=(15,6))
df['state'].value_counts().plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Count of Rows per State')
plt.xlabel('State')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

# Histogram for 'completed_surveys'
plt.figure(figsize=(15,6))
response_merge_states['Completed Surveys'].hist(bins=5, color='skyblue', edgecolor='black')
plt.title('Distribution of Completed Surveys')
plt.xlabel('Completed Surveys')
plt.ylabel('Frequency')
plt.show()

In [ ]:
'''
Dont get rid of test code...
'''

# # data = {
# #     "Completed Surveys": ["FEWER THAN 50","300 or more", "Between 100 and 299",
# #                           "Fewer than 100"]
# # }

# # df = pd.DataFrame(csvfile)

# # # Convert to midpoints
# # def convert_to_midpoint(value):
# #     if value == "300 or more":
# #         return 350  # or a more appropriate higher number if known
# #     elif value == "Between 100 and 299":
# #         return 200

# # response_merge_states["Survey Midpoints"] = response_merge_states["Completed Surveys"].apply(convert_to_midpoint)

# # Bar plot.value_counts().sort_index()
# response_merge_states["Completed Surveys"].plot(kind="bar", color='skyblue')
# plt.xlabel('Survey Categories')
# plt.ylabel('Counts')
# plt.title('Distribution of Completed Surveys')
# # plt.xticks(ticks=[0,1], labels=["FEWER THAN 50","300 or more", "Between 100 and 299",
# #                           "Fewer than 100"], rotation=45)
# plt.show()

# Code for creating files.
Files will be ported to Maven and PowerBI

In [ ]:
#response_merge_states.to_csv('file_name', encoding='utf-8', index=False)

In [ ]:
#sampled_response = response_merge_states.sample(n=100, random_state=42)

In [ ]:
#sampled_response.to_csv('sampled_data',encoding='utf-8',index=False)

In [ ]:
##naturalearth_lowres is depricated and needed to create a new json file for the heatmap.

# Load the GeoJSON
gdf = gpd.read_file("geo_data.json")

# Inspect the first row
print(gdf.iloc[50])

In [ ]:
avg_response = response_merge_states.groupby('State')['Response Rate (%)'].mean()
avg_response

nan_df = avg_response.isna()
print(nan_df)

# m = folium.Map([43,-100],zoom_start=4)

# folium.Choropleth(
#     geo_data="geo_data.json",
#     name='choropleth',
#     data=avg_response,
#     columns=[avg_response.index, avg_response.values],
#     key_on='feature.id',
#     fill_color='YlGn',
#     fill_opacity=0.7,
#     line_opacity=0.2,
#     legend_name='Average Response Rate (%)'
# ).add_to(m)

# m.save('us_states_response_rate.html')

# # df = pd.read_csv(pd.compat.StringIO(data))

# # Average response rate by state
# avg_response = response_merge_states.groupby('State')['Response Rate (%)'].mean()

# # Load the US map data
# us_map = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres')).query('continent == "North America" and name == "United States of America"')
# type(us_map)
# # Prepare data for Folium
# # state_geo = us_map.set_index('name').reindex(avg_response.index)

# # # Convert to a format Folium understands
# # state_json = state_geo.to_json()

# # # Create the map
# # m = folium.Map([43, -100], zoom_start=4)

# # folium.Choropleth(
# #     geo_data=state_json,
# #     name='choropleth',
# #     data=avg_response,
# #     columns=[avg_response.index, avg_response.values],
# #     key_on='properties.name',
# #     fill_color='YlGn',
# #     fill_opacity=0.7,
# #     line_opacity=0.2,
# #     legend_name='Average Response Rate (%)'
# # ).add_to(m)

# m.save('us_states_response_rate.html')
# # # Get the average response rate for each state
# # avg_response_rate = response_merge_states.groupby('State Name')['Response Rate (%)'].mean().reset_index()

# # # Get US states' geometries
# # world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
# # usa = world[world['name'] == "United States of America"].explode().reset_index(drop=True)

# # # Merge on the state name
# # merged = usa.set_index('name').join(avg_response_rate.set_index('State Name'))

# # # Plotting the heatmap using folium
# # m = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

# # # Convert to geojson for folium
# # geo_json_data = merged.to_json()

# # geo_json_data


# # # Add the heatmap data
# # folium.Choropleth(
# #     geo_data=geo_json_data,
# #     name='choropleth',
# #     data=avg_response_rate,
# #     columns=['State Name', 'Response Rate (%)'],
# #     key_on='properties.State Name',
# #     fill_color='YlGn',
# #     fill_opacity=0.7,
# #     line_opacity=0.2,
# #     legend_name='Average Response Rate (%)'
# # ).add_to(m)

# # folium.LayerControl().add_to(m)
# # m.save("heatmap.html")